In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.utils import class_weight
import os

### Configuração da GPU no TensorFlow

Antes de iniciar o treino do modelo, é importante garantir que o *TensorFlow* está configurado para utilizar a GPU (caso esteja disponível). Além disso, ativamos o *memory growth*, que permite ao *TensorFlow* alocar memória da GPU conforme necessário, evitando reservar toda a memória de uma vez.

In [ ]:
# Verificar dispositivos físicos do tipo 'GPU' disponíveis
gpus = tf.config.list_physical_devices('GPU')

# Se houver GPUs disponíveis, configurar o memory growth
if gpus:
    try:
        for gpu in gpus:
            # Ativar crescimento dinâmico da memória da GPU
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        # Caso a GPU já tenha sido inicializada, não é possível alterar a configuração
        print(e)

### Exploração da Estrutura de Diretorias

Antes de carregar os dados, é importante garantir que o caminho para os ficheiros está correto e que a estrutura de diretorias está bem organizada. O seguinte bloco de código permite:

- Definir o caminho base (`root_path`) para o projeto.
- Listar as diretorias existentes na raiz.
- Verificar se o diretório do dataset (`garbage-noaug-70-15-15`) existe e visualizar o seu conteúdo.
- Explorar de forma recursiva a estrutura de diretorias, mostrando ficheiros e pastas com indentação hierárquica.

Este passo é essencial para:
- Validar que os dados estão organizados corretamente.
- Evitar erros de caminho ao carregar imagens para treino, validação e teste.

In [ ]:

# Caminho local para a pasta raiz do projeto
root_path = "./"  

# Listar diretorias no caminho raiz
print("Diretorias no caminho raiz:")
print(os.listdir(root_path))

# Verificar conteúdo de um caminho específico
specific_path = os.path.join(root_path, "garbage-noaug-70-15-15")
if os.path.exists(specific_path):
    print(f"\nConteúdo de {specific_path}:")
    print(os.listdir(specific_path))
else:
    print(f"\nCaminho {specific_path} não existe")

# Função para listar diretorias com profundidade
def list_dirs(path, indent=0):
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        if os.path.isdir(full_path):
            print(" " * indent + "📁 " + item)
            if indent < 4:
                list_dirs(full_path, indent + 2)
        else:
            print(" " * indent + "📄 " + item)

# Explorar estrutura de diretorias
print("\nEstrutura de diretorias:")
list_dirs(root_path, 0)

### Deteção e Configuração Otimizada de GPU (Apple Silicon / Metal)

Este bloco de código trata da deteção e configuração de dispositivos de aceleração como GPUs ou MPS (*Metal Performance Shaders*), especialmente útil em Macs com Apple Silicon.

#### Funcionalidades:
- Procura dispositivos GPU disponíveis (*TensorFlow* ≥ 2.5 reconhece *Metal* como `GPU`).
- Se não encontrar GPU, tenta encontrar dispositivos `MPS` diretamente.
- Ativa `memory growth` para evitar alocação antecipada excessiva de memória.
- Verifica e imprime os dispositivos visíveis.
- Executa uma multiplicação de matrizes simples para testar a aceleração via GPU.

In [ ]:
# Improved Metal GPU detection for Apple Silicon
try:
    # First try looking for GPU devices (newer TF versions label Metal as GPU)
    gpus = tf.config.list_physical_devices('GPU')
    if len(gpus) > 0:
        print(f"Found {len(gpus)} GPU device(s)")
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU acceleration enabled (Metal)")
    # If no GPU found, try looking specifically for MPS devices
    elif hasattr(tf.config, 'list_physical_devices') and len(tf.config.list_physical_devices('MPS')) > 0:
        mps_devices = tf.config.list_physical_devices('MPS')
        tf.config.experimental.set_visible_devices(mps_devices[0], 'MPS')
        print("MPS (Metal) device enabled")
    else:
        print("No GPU or MPS device found, using CPU")
        
    # Verify what device is being used
    print("\nDevice being used:", tf.config.get_visible_devices())
    
    # Test with a simple operation to confirm GPU usage
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
        c = tf.matmul(a, b)
        print("Matrix multiplication result:", c)
        print("GPU test successful!")
except Exception as e:
    print(f"Error setting up GPU: {e}")
    print("Falling back to CPU")

### Mixed Precision Training (FP16)

Este bloco de código ativa o **mixed precision training**, que usa `float16` (FP16) em vez de `float32` (FP32), sempre que possível.

#### Benefícios:
- Maior desempenho em GPUs modernas, como as da arquitetura *Volta, Turing, Ampere* ou *Apple Silicon* com suporte a *Metal*.
- Menor uso de memória, permitindo treinar modelos maiores ou

#### Como funciona:
- Operações matemáticas intensas usam `float16`
- A perda (`loss`) e os pesos principais mantêm-se em `float32` para estabilidade numérica

In [ ]:
# Enable mixed precision (faster on GPU)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')  # Use FP16 instead of FP32


### Carregamento e Preparação dos Dados com Pesos de Classe

Este bloco de código trata da preparação e carregamento dos dados para treino do modelo, incluindo o cálculo de **pesos de classe** para compensar desequilíbrios no *dataset*, bem como otimizações com *prefetching* e *shuffling*. Esta preparação é fundamental para o treino eficaz de modelos baseados em transferência de aprendizagem, como a VGG16.

#### Definição de Caminhos

```python
train_dir = specific_path + "/train"
validation_dir = specific_path + "/valid"
test_dir = specific_path + "/test"
```

Define os diretórios onde se encontram as imagens organizadas por classe. O caminho `specific_path` representa a localização base do *dataset*, e os subdiretórios `train`, `valid` e `test` contêm os dados de treino, validação e teste, respetivamente.

#### Configurações de Imagem

```python
IMG_SIZE = 128
BATCH_SIZE = 16
```

- `IMG_SIZE`: Redimensiona todas as imagens para 128x128, o que permite acelerar o treino e reduzir o consumo de memória, mantendo um nível de detalhe suficiente.
- `BATCH_SIZE`: Um valor mais pequeno (16) é adotado para permitir um treino mais estável e compatível com hardware com menos memória.

#### Carregamento do Dataset e Cálculo de Pesos

```python
train_dataset = tf.keras.utils.image_dataset_from_directory(...)
```

Carrega automaticamente as imagens a partir das subpastas e associa cada imagem ao respetivo rótulo com base no nome da diretoria.

Em seguida, os rótulos são extraídos do *dataset* com:

```python
train_labels = np.concatenate([y.numpy() for x, y in train_dataset], axis=0)
```

e os **pesos de classe** são calculados com:

```python
class_weights = class_weight.compute_class_weight(...)
```

Esta abordagem é essencial quando o *dataset* apresenta **desequilíbrio entre classes**, permitindo que o modelo atribua maior importância às classes menos representadas durante o processo de treino. O dicionário `class_weights` é posteriormente utilizado no método `model.fit()`.

#### Carregamento de Validação e Teste

```python
val_dataset = tf.keras.utils.image_dataset_from_directory(...)
test_dataset = tf.keras.utils.image_dataset_from_directory(...)
```

Os conjuntos de validação e teste são carregados de forma semelhante, mas sem necessidade de cálculo de pesos. Estes conjuntos são usados para monitorizar o desempenho do modelo ao longo do treino e na fase de avaliação final, respetivamente.

#### Otimização com *Shuffle* e *Prefetching*

```python
.shuffle(buffer_size=10).prefetch(buffer_size=AUTO_TUNE)
```

Aplica duas otimizações fundamentais:

- **Shuffle**: embaralha os dados, ajudando a evitar que o modelo aprenda padrões indesejados na ordem dos dados;
- **Prefetch**: carrega batches futuros em paralelo com o treino, reduzindo a latência entre iterações e melhorando a eficiência geral do *pipeline*.

In [ ]:
# Definição das diretorias de treino, validação e teste
train_dir = specific_path + "/train"
validation_dir = specific_path + "/valid"
test_dir = specific_path + "/test"

# Definir o tamanho das imagens e o tamanho do batch
# (Imagens originais têm 640px, mas 128px acelera o treino)
IMG_SIZE = 128
BATCH_SIZE = 16

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE), # Redimensionar imagens
    batch_size=BATCH_SIZE            # Dividir em batches
)

# Extrair rótulos dos batches do dataset
train_labels = np.concatenate([y.numpy() for x, y in train_dataset], axis=0)

# Calcular os pesos das classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Carregar o dataset de validação
val_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Carregar o dataset de teste
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

class_names = train_dataset.class_names

# Aplicar preprocessamento de imagens
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.shuffle(buffer_size=10).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=10).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.shuffle(buffer_size=10).prefetch(buffer_size=AUTOTUNE)

### Preparação do Modelo com Transferência de Aprendizagem

```python
base_model = tf.keras.applications.VGG16(...)
base_model.trainable = False
```

O modelo base escolhido é a **VGG16**, pré-treinada no *dataset ImageNet*. Ao definir `trainable = False`, congela-se a base convolucional, utilizando-a apenas como **extrator de características** (*feature extractor*) nesta primeira fase. Camadas densas personalizadas serão adicionadas no topo para adaptar o modelo à tarefa específica de classificação de resíduos.

Esta abordagem reduz o tempo de treino, evita sobreajuste em datasets pequenos e tira partido do conhecimento previamente aprendido em tarefas visuais genéricas.

In [ ]:
# Feature Extraction – VGG16 congelada
base_model = tf.keras.applications.VGG16(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # congelado inicialmente


### Construção e Compilação do Modelo com Transferência de Aprendizagem (VGG16)

Este bloco de código define a arquitetura completa do modelo com base em **transferência de aprendizagem**, combinando uma rede convolucional pré-treinada (neste caso, VGG16) com camadas densas personalizadas. A base convolucional é utilizada como **extrator de características**, e permanece congelada na fase inicial de treino.

#### Definição da Entrada e Pré-processamento

```python
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)
```

- Define a forma de entrada das imagens (RGB);
- Aplica normalização dos valores de pixel para o intervalo [0, 1], o que melhora a estabilidade do treino.

#### Extrator de Características (Base Pré-treinada)

- A `base_model` (como VGG16) é utilizada com os seus pesos pré-treinados no *ImageNet*;
- É aplicada com `training=False` para manter os seus pesos congelados;
- A camada `GlobalAveragePooling2D` reduz a dimensionalidade, convertendo os mapas de ativação em vetores.

#### Camadas Densas Personalizadas

As camadas densas adicionadas ao topo da rede extraem relações mais complexas entre as características aprendidas:

- `Dropout`: Técnica de regularização para reduzir *overfitting*;
- `Dense`: Camadas totalmente ligadas com 512 e 256 unidades;
- `BatchNormalization`: Normaliza as ativações entre batches, estabilizando o treino;
- `ReLU`: Função de ativação não-linear comum em redes profundas.

#### Camada de Saída

- Camada final com tantos neurónios quanto o número de classes;
- A função `softmax` converte os logits em probabilidades para cada classe.

#### Criação e Compilação do Modelo

- O modelo é instanciado usando a API funcional do Keras;
- O otimizador `Adam` é utilizado com uma taxa de aprendizagem de `0.0003`;
- A função de perda `sparse_categorical_crossentropy` é usada para *targets* inteiros;
- A métrica principal é `accuracy`.

#### Sumário do Modelo

Apresenta um resumo da arquitetura do modelo, incluindo o número de parâmetros treináveis e não treináveis, bem como a estrutura das camadas.

Esta combinação de transferência de aprendizagem com uma cabeça densa personalizada permite aproveitar o poder dos modelos de larga escala, adaptando-os eficazmente à tarefa específica de classificação de resíduos.


In [ ]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Normalização
x = layers.Rescaling(1./255)(inputs)

# Extrator de features (VGG16 congelada)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)

# Camadas densas otimizadas
x = layers.Dropout(0.3)(x)

x = layers.Dense(512)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.4)(x)

x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.4)(x)

# Saída
outputs = layers.Dense(len(class_names), activation='softmax')(x)

# Modelo final
model = keras.Model(inputs, outputs)

# Compilação
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumo
model.summary()


### Treino com Transferência de Aprendizagem e *Fine-Tuning*

Este bloco de código descreve as duas fases principais de treino do modelo baseado em **transferência de aprendizagem com VGG16**, incluindo **feature extraction** inicial e subsequente **fine-tuning** das camadas superiores da rede. São também aplicadas técnicas de *early stopping* e ajuste dinâmico da *learning rate* para melhorar a eficiência e estabilidade do treino.

#### EarlyStopping Inicial

- Monitoriza a métrica de `val_loss`;
- Interrompe o treino se a perda de validação não melhorar durante 5 épocas consecutivas;
- Restaura os pesos da época com melhor desempenho.

#### Fase 1 – Feature Extraction

- O modelo é treinado com a **VGG16 congelada**;
- Apenas as camadas densas adicionadas no topo são ajustadas;
- Utiliza-se o dataset completo e monitoriza-se a validação durante o treino;
- EarlyStopping assegura treino eficiente sem overfitting.

#### Fase 2 – *Fine-Tuning*

- Descongela as **últimas 50 camadas** da VGG16 para permitir o ajuste fino dos pesos;
- As camadas mais antigas permanecem congeladas, preservando o conhecimento genérico aprendido no *ImageNet*.

O modelo é recompilado com uma taxa de aprendizagem reduzida:

- A *learning rate* mais baixa evita alterações bruscas nos pesos durante o *fine-tuning*.

#### *Callbacks* para *Fine-Tuning*

- Um novo `EarlyStopping` com os mesmos parâmetros é aplicado;
- O callback `ReduceLROnPlateau` reduz dinamicamente a *learning rate* se a `val_loss` estagnar.

#### Execução do *Fine-Tuning*

- O treino é realizado com os pesos de classe calculados previamente;
- As melhorias obtidas nesta fase permitem uma melhor adaptação do modelo às especificidades do domínio (resíduos urbanos).

#### Salvamento do Modelo

- Os **pesos do modelo** e o **modelo completo** (arquitetura + pesos + estado do otimizador) são guardados para reutilização futura;
- Permite retomar o treino, realizar inferência ou exportar para produção.

Este *pipeline* estruturado em duas fases permite combinar a robustez de um modelo pré-treinado com a capacidade de especialização para uma tarefa concreta, otimizando tanto o desempenho como o tempo de treino.


In [ ]:
# EarlyStopping mais agressivo
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# 4. Treino com feature extraction
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stopping]
)

# 5. Fine-tuning – descongela últimas camadas
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Novo EarlyStopping para fine-tuning
early_stopping_ft = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Ajuste dinâmico da LR
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)

# Treino com fine-tuning
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stopping_ft, reduce_lr],
    class_weight=class_weights
)

model.save_weights('models/vgg16_noaug.weights.h5')
# Salvar o modelo completo
model.save('models/vgg16_noaug.keras')
print("Modelo salvo como 'vgg16_noaug.keras' e pesos como 'vgg16_noaug.h5'.")  


### Avaliação Final do Modelo e Análise de Desempenho

Após o treino e *fine-tuning* do modelo, realiza-se a **avaliação final** no conjunto de teste. Esta etapa permite medir a capacidade do modelo para generalizar perante dados nunca vistos, e avaliar quantitativamente e qualitativamente os seus erros e acertos.

#### Avaliação no Conjunto de Teste

- O método `evaluate` calcula a **loss** e **accuracy** do modelo sobre o conjunto de teste;
- A `test accuracy` representa a percentagem de classificações corretas;
- A `test loss` quantifica o erro médio cometido pelo modelo.

#### Previsões e Comparação com Valores Reais

- `y_pred`: Armazena as classes previstas pelo modelo (via `argmax`);
- `y_true`: Contém os rótulos reais;
- Esta informação é essencial para gerar métricas adicionais além da *accuracy*.

#### Relatório de Classificação

Gera um relatório com as seguintes métricas por classe:

- **Precision**: proporção de previsões corretas entre todas as previsões para uma classe;
- **Recall**: proporção de previsões corretas entre todos os exemplos reais dessa classe;
- **F1-score**: média harmónica entre precision e *recall*;
- **Support**: número de ocorrências reais da classe no conjunto de teste.

Este relatório fornece uma visão detalhada do desempenho do modelo em cada categoria de resíduos.

#### Matriz de Confusão

- A **matriz de confusão** permite visualizar os erros cometidos por classe;
- Cada célula `[i][j]` representa o número de exemplos da classe `i` que foram classificados como `j`;
- A diagonal principal representa acertos — quanto mais dominante, melhor o desempenho;
- Erros sistemáticos podem indicar confusão entre classes visualmente semelhantes (ex: metal vs plástico).

Esta análise é crucial para identificar padrões de erro, avaliar a robustez do modelo em cenários reais e guiar melhorias futuras na arquitetura, dados ou estratégias de treino.


In [ ]:
# Avaliação
test_loss, test_acc = model.evaluate(test_dataset)
print("Test accuracy:", test_acc)

# Previsões para métricas
y_pred = []
y_true = []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())

# Relatório e Confusion Matrix
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

## Visualização da Evolução do Treino

Para uma análise mais clara e interpretável do comportamento do modelo durante o treino, foi gerado um gráfico de dupla visualização com os dados recolhidos a partir do histórico (`history`) fornecido pelo método `model.fit()`.

### Conteúdo Visualizado

O gráfico apresenta duas métricas fundamentais, tanto para os dados de treino como de validação:

1. **Accuracy**:
   - Mostra a proporção de previsões corretas realizadas pelo modelo.
   - Indicador direto da eficácia do modelo em classificar corretamente os exemplos.

2. **Loss**:
   - Representa o valor da função de perda, indicando o quão bem o modelo está a ajustar-se aos dados.
   - Quanto menor o valor, melhor o desempenho do modelo (em teoria).

### Objetivo

Esta visualização tem como objetivo:

- Identificar sinais de **overfitting** (quando a *accuracy* de treino continua a aumentar mas a de validação estagna ou diminui);
- Confirmar a **convergência** do modelo (quando tanto a *loss* como a *accuracy* estabilizam);
- Ajudar a determinar o número ideal de épocas (epochs) para treino.

### Interpretação

- Um **comportamento ideal** é caracterizado por curvas de treino e validação que convergem e permanecem relativamente próximas.
- Se a `validation loss` começar a aumentar enquanto a `training loss` diminui, pode indicar **overfitting**.
- Um bom alinhamento entre `training accuracy` e `validation accuracy` sugere que o modelo está a generalizar bem para dados nunca vistos.

Estes gráficos fornecem, portanto, uma ferramenta essencial de diagnóstico e são altamente recomendados como parte integrante de qualquer processo de treino de redes neuronais.


In [ ]:
# Corrected plotting code for newer TensorFlow versions
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(accuracy) + 1)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy, 'bo-', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'ro-', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Training loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Avaliação Final no Conjunto de Teste

Após o treino e validação do modelo, foi realizada a **avaliação final no conjunto de teste**. Esta etapa é essencial para medir a capacidade do modelo de generalizar para dados completamente novos, que não foram utilizados nem durante o treino nem na validação.

### Métricas Obtidas

A avaliação produz duas métricas principais:

- **Test Accuracy** (`test_acc`): Representa a percentagem de classificações corretas no conjunto de teste.
- **Test Loss** (`test_loss`): Indica o valor da função de perda nesse conjunto, permitindo compreender se o modelo ainda apresenta erros substanciais.

Estes valores fornecem uma estimativa objetiva do desempenho real do modelo em produção.

### Visualização de Previsões

Para uma análise qualitativa, foi implementada uma visualização de **24 imagens aleatórias do conjunto de teste** juntamente com as suas **previsões**. Esta abordagem tem os seguintes objetivos:

- Observar se o modelo é capaz de generalizar para imagens reais com variações visuais e de iluminação;
- Identificar **casos corretos** e **erros de classificação**;
- Avaliar a coerência visual das previsões em relação à classe verdadeira.

#### Detalhes da Visualização:

- Para cada imagem, são apresentados:
  - **True**: A classe real;
  - **Pred**: A classe predita pelo modelo.
- O título de cada imagem é colorido:
  - **Verde**: Previsão correta;
  - **Vermelho**: Previsão incorreta.
- As previsões são obtidas através do método `model.predict`, e a classe final é extraída com `argmax`.

### Importância da Análise Qualitativa

Apesar das métricas globais fornecerem uma visão estatística do desempenho, esta análise visual permite:

- Entender **quais tipos de objetos são mais difíceis de classificar**;
- Verificar **padrões de erro recorrentes**, como confusão entre vidro e plástico ou entre papel e cartão;
- Apoiar a decisão sobre possíveis melhorias no modelo ou na preparação dos dados.

Este tipo de visualização é, portanto, fundamental para interpretar os resultados do modelo no contexto da aplicação real — neste caso, uma app de reconhecimento automático de resíduos através da câmara do dispositivo.


In [ ]:
# Evaluate on test dataset
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Use the already defined class_names variable
print("Classes:", class_names)

# Function to show predictions for a batch of images
plt.figure(figsize=(12, 12))
for images, labels in test_dataset.take(1):
    predictions = model.predict(images)
    pred_classes = np.argmax(predictions, axis=1)
    num_images = images.shape[0]
    grid_rows = int(np.ceil(num_images / 4))
    for i in range(num_images):
        plt.subplot(grid_rows, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        correct = labels[i] == pred_classes[i]
        color = "green" if correct else "red"
        
        plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[pred_classes[i]]}", 
                 color=color)
        plt.axis("off")
plt.tight_layout()
plt.show()